In [155]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from sklearn.model_selection import train_test_split as scikit_train_test_split
from surprise import Reader, Dataset, accuracy
from surprise import KNNBasic, KNNWithMeans, SVD, CoClustering, SlopeOne, SVDpp, NMF, BaselineOnly
from surprise.model_selection import KFold, RepeatedKFold, cross_validate, GridSearchCV, LeaveOneOut
from surprise.model_selection import train_test_split as surprise_train_test_split
from sklearn.pipeline import Pipeline
import time

In [91]:
df = pd.read_csv("person-skills_2022-06-27.csv",sep=";")

Declare various aiding variables

In [92]:
persons = df["person"].unique()
skills = sorted(df["skill"].unique())
categories = df["category"].unique()

In [89]:
employee_dict = {}
for n in range(1,len(persons)+1):
    employee_dict[n] = persons[n-1]

In [90]:
skills_to_category = pd.read_csv("mitarbeiterportal-skills_2022-06-21.csv",sep=";",header=None)
category_dict = dict(zip(skills_to_category[0],skills_to_category[1]))

Insert ratings for skills according to frequency of underlying categories in the skill protfolio on an employee: 

In [ ]:
def scaler_1_5(x,old_max,old_min):
    OldRange = (old_max - old_min)  
    if (OldRange == 0):
        return 1
    else:
        NewRange = (5 - 1)  
        return round((((x - 1) * NewRange) / OldRange) + 1)

In [113]:
# create dataframe for each employee and their category count
category_per_employee = pd.DataFrame(index=categories)
for p in persons:
    p_skills = df[df["person"] == p]
    # count categories for each employee
    category_count = p_skills["category"].value_counts()
    category_count = category_count.apply(lambda x:scaler_1_5(x,category_count.max(),category_count.min()))
    skill_values = np.array([])
    # add count for each category to an array and 0 if category wasn't found (and therefore is not part of resptive employee's skillset)
    for cat in categories:
        try:
            skill_values = np.append(skill_values,category_count[cat])
        except KeyError:
            skill_values = np.append(skill_values,0)
    # add employee data to dataframe
    p_skills_df = pd.DataFrame(index=categories,columns=[p],data=skill_values)
    category_per_employee = pd.concat([category_per_employee,p_skills_df],axis=1)

In [119]:
df_rated = pd.DataFrame()
for p in persons:
    # get employee
    df_subset = df[df["person"] == p].copy()
    # get skills of employee
    p_skills = df_subset["skill"].to_numpy()
    # initialize empty array for ratings
    ratings = np.zeros(len(p_skills))
    # fill array for each skill with respective category value 
    for i in range(0,len(p_skills)): 
        ratings[i] = category_per_employee.loc[category_dict[p_skills[i]]][p]
    # append ratings to subset and drop category column
    df_subset["rating"] = ratings
    df_subset.drop("category",axis=1,inplace=True)
    # append user to df
    df_rated = pd.concat([df_rated,df_subset],ignore_index=True)

In [120]:
df_rated

,person,skill,rating
0,12,Windows NT/2000/XP,4.0
1,12,MAC OS X,4.0
2,12,Windows 7,4.0
3,12,Windows 8,4.0
4,12,iOS,4.0
...,...,...,...
24581,1488,Deutsch,1.0
24582,1488,Spring-JPA,5.0
24583,1488,Maven,3.0
24584,1488,Subversion,3.0


## Comparing Different Recommender Systems

In [235]:
cv_data,holdout_data = scikit_train_test_split(df_rated, test_size=0.05)
reader = Reader(rating_scale=(1,5)) 
data = Dataset.load_from_df(cv_data,reader)
holdout = Dataset.load_from_df(holdout_data,reader)

In [236]:
NUM_TRIALS = 5
NUM_OUTER_SPLITS = 5

COMMENT: Hyperparameter optimization using GridSearchCV can not be incorporated in the cross validation. Doing this yields an error. Multiple work arounds have been unsuccefully tried. As it seems, surprise's GridSearchCV is not compatible with splitting functions such as KFold/LeaveOneOut etc. as well as cross_validate.

In [296]:
def nested_cv(algo,data):
    start = time.time()
    rmse = np.zeros((NUM_TRIALS, NUM_TRIALS))
    mae = np.zeros((NUM_TRIALS, NUM_TRIALS))
    mse = np.zeros((NUM_TRIALS, NUM_TRIALS))
    fit_times = np.zeros((NUM_TRIALS, NUM_TRIALS))
    test_times = np.zeros((NUM_TRIALS, NUM_TRIALS))

    # hier eventuell gridsearch rein? 
    for i in range(NUM_TRIALS):
        cv= LeaveOneOut(n_splits=NUM_OUTER_SPLITS,
                                random_state=36,
                                min_n_ratings=1)
                                # COMMENT: min_n_ratings -> what's the impact of this parameter? 
        cv_results = cross_validate(algo=algo,
                                    data=data,
                                    measures=["rmse","mae","mse"],
                                    cv=cv,
                                    n_jobs=8)
        rmse[i] = cv_results["test_rmse"]
        mae[i] = cv_results["test_mae"]
        mse[i] = cv_results["test_mse"]
        fit_times[i] = cv_results["fit_time"]
        test_times[i] = cv_results["test_time"]

    return mse, rmse, mae, fit_times, test_times

In [297]:
def add_result(results, name, mse, rmse, mae, fit_times, test_times):
    '''
    Function adding the results returned by nested_cv to a dataframe. 
    Results will be aggregated for better comparison. 
    Parameters: 

    Output:
    results DataFrame containing an additional row
    '''
    row = pd.DataFrame({
        "name":name,
        "mse_mean":mse.mean(), 
        "mse_std":mse.std(),
        "rmse_mean":rmse.mean(), 
        "rmse_std":rmse.std(),  
        "mae_mean":mae.mean(), 
        "mae_std":mae.std(), 
        "fit_time":fit_times.mean(),
        "test_time":test_times.mean()
        },index=[0])
    return pd.concat([results,row],ignore_index=True)

In [298]:
results = pd.DataFrame()

In [299]:
algo_list = [
                (BaselineOnly(),"Baseline"),
                (KNNBasic(),"k-NN"),
                (KNNWithMeans(),"Centered k-NN"),
                (SVD(),"SVD"),
                (SVDpp(),"SVD++"),
                (CoClustering(),"CoClustering"),
                (SlopeOne(),"SlopeOne"),
                (NMF(),"NMF")
            ]

In [300]:
for algo in algo_list:
    mse, rmse, mae, fit_times, test_times = nested_cv(algo[0],data)
    results = add_result(results,algo[1],mse,rmse,mae,fit_times,test_times)

In [275]:
results

,name,mse_mean,mse_std,rmse_mean,rmse_std,mae_mean,mae_std,fit_time,test_time
0,Baseline,1.295314,0.040063,1.137981,0.017696,0.926852,0.023953,0.007140,0.001142
1,k-NN,0.890708,0.047954,0.943434,0.025314,0.687647,0.017081,0.035202,0.019591
2,Centered k-NN,0.903997,0.017033,0.950746,0.008911,0.666194,0.014789,0.040716,0.021364
3,SVD,0.917527,0.027022,0.957772,0.014104,0.728502,0.019379,0.100065,0.001162
4,SVD++,0.706811,0.027310,0.840563,0.016258,0.604882,0.009335,1.683412,0.013134
5,CoClustering,1.115214,0.062971,1.055611,0.029988,0.776495,0.031730,0.212278,0.000922
6,SlopeOne,1.113160,0.041171,1.054880,0.019692,0.797153,0.020390,0.033976,0.008588
7,NMF,0.700790,0.056146,0.836449,0.033819,0.576611,0.019858,0.136873,0.000919


Pick out best algorithms and optimize parameters

In [315]:
param_grid = {
                "n_factors": [10, 15, 20,40],
                "n_epochs": [20, 40, 80, 120],
                "biased":[True,False],
                "random_state":[36]
            }
gs = GridSearchCV(NMF, param_grid, measures=["rmse"], cv=3)

gs.fit(data)
# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.939230286078028
{'n_factors': 10, 'n_epochs': 80, 'biased': True, 'random_state': 36}


In [323]:
algo = gs.best_estimator["rmse"]
trainset = data.build_full_trainset()
algo.fit(trainset)

In [324]:
holdout_test = holdout_data.to_numpy()
algo.test(holdout_test)

[Prediction(uid=43, iid='GIT', r_ui=2.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=1488, iid='JSP', r_ui=5.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=494, iid='Objektrelationale Abbildung (ORM)', r_ui=2.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=1227, iid='Unit Tests (Komponententests)', r_ui=3.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=713, iid='arc42', r_ui=3.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=1475, iid='iOS', r_ui=5.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=171, iid='CVS', r_ui=2.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=928, iid='Windows 7', r_ui=4.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=1165, iid='Fortran', r_ui=4.0, est=3.52986, details={'was_impossible': False}),
 Prediction(uid=1452, iid='Unit Tests (Komponententests)', r_ui=5.0, est=3.52986, details={'was_impossible': F

In [220]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [221]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
recommendations = pd.DataFrame()

for uid, user_ratings in top_n.items():
    recom = pd.DataFrame(index=[uid],data=[[iid for (iid, _) in user_ratings]])
    #print([iid for (iid, _) in user_ratings])
    recommendations = pd.concat([recommendations,recom])

recommendations

,0,1,2,3,4,5,6,7,8,9
1032,Redux,"Layouting (Flexbox, CSS Grid)",Bootstrap,RxJS,Zend,SCSS,Browser Debugging,React,xtCommerce,ITCSS
1370,Bootstrap,osCommerce,TypeScript,ePages,Angular Material,Node.js,Cypress,NGRX,Styled Components,ITCSS
1205,Silverlight,ASP.NET MVC-Framework,ASP.NET,Sharepoint Entwicklung,.NET Core,Windows Presentation Foundation (WPF),MSBuild,Windows Communication Foundation (WCF),ASP Generalist,Castor
1194,Windows Server 2008 R2 /2012,OS/2,Novell,Cypress,Sun Solaris,Windows Phone 7,Windows Server 2012 R2,Android,iOS,Symbian
1176,Vue.js,Java ME,Spring-JPA,Jakarta Commons,Oracle XDK,Tailwind CSS,"Layouting (Flexbox, CSS Grid)",Struts (bis Version 1.3),Castor,RxJS
...,...,...,...,...,...,...,...,...,...,...
872,GEF,ITCSS,UX Beratung,Produktstrategie,Browser Debugging,SAP Cloud Platform Integration (SCPI),Web Components,Java FX,Tailwind CSS,Security
64,Oracle XDK,Castor,TopLink,Tailwind CSS,Yii,Xerces,Windows Identity Foundation (WIF),GEF,Swing/AWT,Java FX
556,Castor,BEM - Block Element Modifier,ITCSS,Yii,Xerces,Java FX,Spring-Batch,Browser Debugging,RxJS,"Layouting (Flexbox, CSS Grid)"
279,Infragistics,.NET Core,Kendo UI,Castor,Xerces,Entity Framework,TopLink,Oracle XDK,WebDynpro,Windows Identity Foundation (WIF)


In [234]:
train,test = surprise_train_test_split(holdout,test_size=0.99)
holdout_preds = algo.test(test)
accuracy.rmse(holdout_preds)

RMSE: 0.5353


0.5353351018056478